In [1]:
import requests
import pandas as pd
import pprint

In [2]:
keytrans = {
    'id': 'serieid',
    'uom': 'serieunits',
    'station.properties.id': 'siteid',
    'station.properties.label': 'sitekey',
    'station.geometry.coordinates': 'geom',
    'firstValue.timestamp': 'start',
    'lastValue.timestamp': 'stop',
    #'parameters.feature.id': 'featureid',
    #'parameters.feature.label': 'featurekey',
    'parameters.phenomenon.id': 'measureid',
    'parameters.phenomenon.label': 'measurekey',
    #'parameters.category.id': 'categoryid',
    #'parameters.category.label': 'categorykey',
    #'statusIntervals': 'status'
}

In [3]:
def getMeta():
    # Download Timeserie Metadata:
    url = "https://geo.irceline.be/sos/api/v1/{target:}/?expanded=true"
    rep = requests.get(url.format(target='timeseries'))
    df = pd.io.json.json_normalize(rep.json())
    # Clean columns:
    df = df.loc[:,keytrans.keys()].rename(columns=keytrans)
    # Casts & postprocessing
    df['start'] = pd.to_datetime(df['start'], origin='unix', unit='ms')
    df['stop'] = pd.to_datetime(df['stop'], origin='unix', unit='ms')
    df['sitename'] = df['sitekey'].apply(lambda x: "-".join(x.split('-')[1:]))
    df['sitekey'] = df['sitekey'].apply(lambda x: x.split('-')[0].strip())
    df['lat'] = df['geom'].apply(lambda x: x[0])
    df['lon'] = df['geom'].apply(lambda x: x[1])
    return df.loc[:,['serieid', 'siteid', 'measureid', 'serieunits', 'measurekey', 'sitekey',
                     'sitename', 'lat', 'lon', 'start', 'stop']]

In [ ]:
meta = getMeta()
meta.sample(5)